# Purpose of This Project
This project will contain python scripts for the purposes of simulating and analyzing GURPS 4e combat mechanics. I plan to take a functional approach to this calculator to practice my functional programming in python. I may or may not break down and cry and write for loops instead. I expect to embarrassingly mess up the whole adhering to the functional paradigm entirely and have to revisit a lot of my functions. Either way, I'll write it all down in a Jupyter notebook to document the journey.

First, I'll start by finding a function that can roll dice. GURPS 4e combat is highly reliant on dice rolls, so—while we can analyze the rolls mathematically—the ability to run simulations should be a lot more practical in situations where we are learning the mathematics or want to see how it might feel to a player in our game.

We'll use the random module

In [1]:
import random as rd

print(rd.randint(3,18))

10


GURPS 4e heavily utilizes the roll of three six sided dice (3d6), abbreviated in the context of the system simply to 3d. Our 3d rolls will have a minimum of 3 and a maximum of 18, as shown above, but the distribution won't be completely linear. We might repackage the `rd.randint()` function into another function. On that note, regarding the functional approach I'd like to take, I am not going to remake `rd.randint()` from scratch to make it more "functional". This approach practice will only apply to user-defined functions for the time being.

First, we should aim to make a couple of functions: one to roll any number of dice and output a number, particularly rolls of 3d, and one that calculates the probability of getting any given number from a roll.

We are going to assume that every single dice we work with will be one that starts at 1, and counts up one whole number until there is a unique number on every face of an n-sided die from 1 to n.

# Dice Roll Functions

In [2]:
def roll_dx(x):
    return rd.randint(1,x)

def roll_xd6(x):
    return x*roll_dx(6)

print(f'roll 3d6 5 times: {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}')
print(f'roll 5d6 5 times: {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}')

roll 3d6 5 times: 3, 12, 15, 12, 3
roll 5d6 5 times: 30, 15, 10, 5, 30


The way that our `roll_dx()` function is written, it will give an error for any input besides whole numbers greater than or equal to `1`.

# Dice Probability Functions

Our first dice probability function will be a lot like our function `roll_dx` above. The input for the function will be a whole number greater than or equal to `1` that says how many sides we have on our dice. The output will be a dictionary of all the possible outcomes as keys and their probabilities as values.

## Coming up with a solution
the first approach that comes to mind is to take our aforementioned input as a local variable `x`, then create an empty dictionary and to add entries to. In the case of an input that is `6`, we would want our function to add numbers from 1 to 6 to our dictionary, and make the value a probability in the form of a calculation: 1/6 for each one.

I'm doing some lame empty dict technique for this, so it may seem like I have given up on making this hardcore functional and you are correct. The reason is that I have a baby to take care of and she doesn't have the patience for dad to learn how to do this. Refactoring our functions will come at a later date.

In [3]:
def prob_dx(x):
    d = {}
    for i in range(x):
        d[i+1] = 1/x
    return d

print(prob_dx(6))

{1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666}
